# Meta data

In [1]:
from dataclasses import dataclass
from typing import Literal, Optional

Rect = tuple[int,int,int,int]

@dataclass
class Meta:
    width: int; height: int; fps: int
    title_safe: float; action_safe: float

@dataclass
class MotionSpec:
    enter_type: Literal["fade","slide-up","none"] = "fade"
    enter_dur: float = 0.25
    exit_type: Literal["fade","none"] = "fade"
    exit_dur: float = 0.20
    delay: float = 0.0

@dataclass
class Slot:
    slot_id: str
    rect: Rect
    align: Literal["left","center","right"] = "left"
    text: Optional[str] = None
    items: Optional[list[str]] = None
    font: str = "Inter"
    size: int = 32
    weight: int = 700
    color: str = "#ffffff"
    motion: Optional[MotionSpec] = None
    layer: int = 10

@dataclass
class GraphicSpec:
    src: str
    rect: Rect
    mode: Literal["fit","cover"] = "fit"
    caption: Optional[str] = None
    layer: int = 5
    motion: Optional[MotionSpec] = None

@dataclass
class PresenterSpec:
    src: str
    rect: Rect
    shape: Literal["circle","rect"] = "circle"
    layer: int = 20

@dataclass
class Scene:
    id: str; type: str; start: float; duration: float
    background: dict  # {"color": "..."} | {"video": "..."} | {"image": "..."}
    slots: list[Slot]
    graphics: list[GraphicSpec]
    presenter: Optional[PresenterSpec] = None
    transition_out: dict | None = None

@dataclass
class Timeline:
    meta: Meta
    scenes: list[Scene]


# I/O Validation

In [2]:
# io_validation.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple, Optional, Literal
import json, os

Rect = Tuple[int, int, int, int]

# ---------- Data classes (tối thiểu để validate / report) ----------
@dataclass
class ValidationIssue:
    level: Literal["ERROR", "WARN"]     # mức độ
    where: str                          # đường dẫn logic: meta.canvas, scene[03].slots[heading_h1]...
    code: str                           # mã ngắn để lọc
    msg: str                            # diễn giải

def _ok(x: bool) -> bool: return bool(x)

# ---------- 1) load_timeline ----------
def load_timeline(json_path: str) -> Dict[str, Any]:
    """
    Đọc file JSON timeline (utf-8), trả về dict.
    Raise FileNotFoundError / json.JSONDecodeError nếu lỗi.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        return json.load(f)

# ---------- 2) validate_timeline ----------
def validate_timeline(tl: Dict[str, Any]) -> List[ValidationIssue]:
    """
    Kiểm tra cấu trúc & các ràng buộc quan trọng:
    - meta.canvas (width/height/fps), safe_margins 0..0.2
    - timeline[]: scene có id/type/start/duration > 0
    - scene thời gian tăng dần (không bắt buộc liên tiếp), không âm
    - background tối thiểu (color|image|video)
    - slots[]: rect trong canvas, align hợp lệ, style.size trong [8..200]
    - graphics[]: rect + mode {fit,cover}; caption optional
    - presenter: shape {circle,rect}, rect hợp lệ
    - transition_out: type {cut,dissolve}, dur ≥ 0 (nếu có)
    """
    issues: List[ValidationIssue] = []

    # ---- meta ----
    meta = tl.get("meta")
    if not isinstance(meta, dict):
        issues.append(ValidationIssue("ERROR", "meta", "META_MISSING", "Thiếu khối meta"))
        return issues

    canvas = meta.get("canvas", {})
    width = canvas.get("width")
    height = canvas.get("height")
    fps = canvas.get("fps", 30)

    if not isinstance(width, int) or width <= 0:
        issues.append(ValidationIssue("ERROR", "meta.canvas.width", "CANVAS_WIDTH", "width phải là số nguyên dương"))
    if not isinstance(height, int) or height <= 0:
        issues.append(ValidationIssue("ERROR", "meta.canvas.height", "CANVAS_HEIGHT", "height phải là số nguyên dương"))
    if not (isinstance(fps, int) and fps > 0):
        issues.append(ValidationIssue("ERROR", "meta.canvas.fps", "CANVAS_FPS", "fps phải là số nguyên dương"))

    safe = meta.get("safe_margins", {})
    title_area = safe.get("title_area", 0.05)
    action_area = safe.get("action_area", 0.025)
    if not (isinstance(title_area, (int, float)) and 0 <= title_area <= 0.2):
        issues.append(ValidationIssue("WARN", "meta.safe_margins.title_area", "SAFE_TITLE", "title_area nên trong [0..0.2]"))
    if not (isinstance(action_area, (int, float)) and 0 <= action_area <= 0.2):
        issues.append(ValidationIssue("WARN", "meta.safe_margins.action_area", "SAFE_ACTION", "action_area nên trong [0..0.2]"))

    # Chuẩn hoá thông số dùng nhiều
    cw = width if isinstance(width, int) else 1920
    ch = height if isinstance(height, int) else 1080

    # ---- timeline ----
    scenes = tl.get("timeline")
    if not isinstance(scenes, list) or not scenes:
        issues.append(ValidationIssue("ERROR", "timeline", "TIMELINE_EMPTY", "Thiếu danh sách scene"))
        return issues

    # kiểm tra trùng id
    seen_ids = set()
    prev_start = -1.0
    for i, sc in enumerate(scenes):
        where_sc = f"timeline[{i:02d}]"
        sid = sc.get("id")
        stype = sc.get("type")
        start = sc.get("start")
        dur = sc.get("duration")

        if not sid or not isinstance(sid, str):
            issues.append(ValidationIssue("ERROR", f"{where_sc}.id", "SC_ID", "Scene phải có id (string)"))
        else:
            if sid in seen_ids:
                issues.append(ValidationIssue("ERROR", f"{where_sc}.id", "SC_ID_DUP", f"Trùng scene id: {sid}"))
            seen_ids.add(sid)

        if not stype or not isinstance(stype, str):
            issues.append(ValidationIssue("ERROR", f"{where_sc}.type", "SC_TYPE", "Scene phải có type (string)"))

        if not isinstance(start, (int, float)) or start < 0:
            issues.append(ValidationIssue("ERROR", f"{where_sc}.start", "SC_START", "start phải là số ≥ 0"))
        if not isinstance(dur, (int, float)) or dur <= 0:
            issues.append(ValidationIssue("ERROR", f"{where_sc}.duration", "SC_DUR", "duration phải > 0"))

        if isinstance(start, (int, float)) and start < prev_start:
            issues.append(ValidationIssue("WARN", f"{where_sc}.start", "SC_ORDER", "start nhỏ hơn scene trước (xem lại thứ tự thời gian)"))
        prev_start = start if isinstance(start, (int, float)) else prev_start

        # background
        bg = sc.get("background", {})
        if not isinstance(bg, dict) or not any(k in bg for k in ("color", "image", "video")):
            issues.append(ValidationIssue("WARN", f"{where_sc}.background", "SC_BG", "Nên có background color/image/video"))

        # transition_out
        trans = sc.get("transition_out")
        if trans is not None:
            if not isinstance(trans, dict):
                issues.append(ValidationIssue("ERROR", f"{where_sc}.transition_out", "SC_TR_TYPE", "transition_out phải là object"))
            else:
                ttype = trans.get("type", "cut")
                if ttype not in ("cut", "dissolve"):
                    issues.append(ValidationIssue("WARN", f"{where_sc}.transition_out.type", "SC_TR_KIND", "transition_out.type nên là 'cut'|'dissolve'"))
                tdur = trans.get("dur", 0.0)
                if not isinstance(tdur, (int, float)) or tdur < 0:
                    issues.append(ValidationIssue("WARN", f"{where_sc}.transition_out.dur", "SC_TR_DUR", "transition_out.dur nên ≥ 0"))

        # slots
        slots = sc.get("slots", [])
        if slots and not isinstance(slots, list):
            issues.append(ValidationIssue("ERROR", f"{where_sc}.slots", "SLOTS_TYPE", "slots phải là list"))
        for j, s in enumerate(slots or []):
            where_sl = f"{where_sc}.slots[{j}]"
            _validate_slot(s, where_sl, issues, cw, ch)

        # graphics
        gfxs = sc.get("graphics", [])
        if gfxs and not isinstance(gfxs, list):
            issues.append(ValidationIssue("ERROR", f"{where_sc}.graphics", "GFX_TYPE", "graphics phải là list"))
        for j, g in enumerate(gfxs or []):
            where_g = f"{where_sc}.graphics[{j}]"
            _validate_graphic(g, where_g, issues, cw, ch)

        # presenter
        pres = sc.get("presenter")
        if pres is not None:
            _validate_presenter(pres, f"{where_sc}.presenter", issues, cw, ch)

    return issues

def _validate_slot(s: Dict[str, Any], where: str, issues: List[ValidationIssue], cw: int, ch: int):
    sid = s.get("slot_id")
    if not isinstance(sid, str) or not sid:
        issues.append(ValidationIssue("ERROR", f"{where}.slot_id", "SLOT_ID", "Thiếu slot_id (string)"))
    layout = s.get("layout", {})
    if not isinstance(layout, dict):
        issues.append(ValidationIssue("ERROR", f"{where}.layout", "SLOT_LAYOUT", "Thiếu layout object"))
        return
    rect = layout.get("rect")
    align = layout.get("align", "left")
    if not _rect_is_valid(rect, cw, ch):
        issues.append(ValidationIssue("ERROR", f"{where}.layout.rect", "RECT", f"rect không hợp lệ hoặc vượt canvas: {rect}"))
    if align not in ("left", "center", "right"):
        issues.append(ValidationIssue("WARN", f"{where}.layout.align", "ALIGN", "align nên là left|center|right"))

    style = s.get("style", {})
    if not isinstance(style, dict):
        issues.append(ValidationIssue("WARN", f"{where}.style", "STYLE_OBJ", "style nên là object"))
    else:
        size = style.get("size", 32)
        if not (isinstance(size, (int, float)) and 8 <= size <= 200):
            issues.append(ValidationIssue("WARN", f"{where}.style.size", "FONT_SIZE", "size nên trong [8..200] px"))
        weight = style.get("weight", 700)
        if not (isinstance(weight, int) and 100 <= weight <= 900):
            issues.append(ValidationIssue("WARN", f"{where}.style.weight", "FONT_WEIGHT", "weight nên trong [100..900]"))

def _validate_graphic(g: Dict[str, Any], where: str, issues: List[ValidationIssue], cw: int, ch: int):
    src = g.get("src")
    if not isinstance(src, str) or not src:
        issues.append(ValidationIssue("ERROR", f"{where}.src", "GFX_SRC", "Thiếu đường dẫn graphic src"))
    layout = g.get("layout", {})
    if not isinstance(layout, dict):
        issues.append(ValidationIssue("ERROR", f"{where}.layout", "GFX_LAYOUT", "Thiếu layout object"))
        return
    rect = layout.get("rect")
    if not _rect_is_valid(rect, cw, ch):
        issues.append(ValidationIssue("ERROR", f"{where}.layout.rect", "RECT", f"rect không hợp lệ hoặc vượt canvas: {rect}"))
    mode = layout.get("mode", "fit")
    if mode not in ("fit", "cover"):
        issues.append(ValidationIssue("WARN", f"{where}.layout.mode", "GFX_MODE", "mode nên là 'fit' hoặc 'cover'"))
    # caption optional => không bắt buộc kiểm tra

def _validate_presenter(p: Dict[str, Any], where: str, issues: List[ValidationIssue], cw: int, ch: int):
    src = p.get("src")
    if not isinstance(src, str) or not src:
        issues.append(ValidationIssue("ERROR", f"{where}.src", "PIP_SRC", "Thiếu presenter src"))
    rect = p.get("rect")
    if not _rect_is_valid(rect, cw, ch):
        issues.append(ValidationIssue("ERROR", f"{where}.rect", "RECT", f"rect không hợp lệ hoặc vượt canvas: {rect}"))
    shape = p.get("shape", "circle")
    if shape not in ("circle", "rect"):
        issues.append(ValidationIssue("WARN", f"{where}.shape", "PIP_SHAPE", "shape nên là 'circle' hoặc 'rect'"))

def _rect_is_valid(rect: Any, cw: int, ch: int) -> bool:
    if not (isinstance(rect, (list, tuple)) and len(rect) == 4):
        return False
    x, y, w, h = rect
    if not all(isinstance(v, (int, float)) for v in (x, y, w, h)): return False
    if w <= 0 or h <= 0: return False
    # Cho phép vượt nhẹ safe-zone nhưng không vượt canvas
    if x < 0 or y < 0: return False
    if x + w > cw + 0.001 or y + h > ch + 0.001: return False
    return True

# ---------- 3) collect_assets ----------
def collect_assets(tl: Dict[str, Any]) -> Dict[str, Dict[str, Any]]:
    """
    Duyệt toàn bộ scenes để gom asset paths:
    return {"images":{path:exists_bool}, "videos":{path:exists_bool}}
    * Bỏ qua data URI (bắt đầu bằng "data:")
    """
    res = {"images": {}, "videos": {}}
    def _add(path: Optional[str], bucket: str):
        if not path or not isinstance(path, str): return
        if path.startswith("data:"): return
        res[bucket][path] = os.path.exists(path)

    for sc in tl.get("timeline", []) or []:
        bg = sc.get("background", {})
        if "image" in bg: _add(bg.get("image"), "images")
        if "video" in bg: _add(bg.get("video"), "videos")

        for g in sc.get("graphics", []) or []:
            _add(g.get("src"), "images")  # giả định graphic là ảnh; nếu có video graphic thì đổi bucket theo thực tế

        pres = sc.get("presenter")
        if pres:
            # presenter thường là video/ảnh
            path = pres.get("src")
            if path and isinstance(path, str) and not path.startswith("data:"):
                ext = os.path.splitext(path)[1].lower()
                if ext in (".mp4", ".mov", ".mkv", ".webm"):
                    _add(path, "videos")
                else:
                    _add(path, "images")
    return res

# ---------- Helper: pretty report ----------
def format_issues(issues: List[ValidationIssue]) -> str:
    if not issues:
        return "✅ No validation issues."
    lines = []
    # ERROR trước, rồi WARN
    for lvl in ("ERROR", "WARN"):
        group = [i for i in issues if i.level == lvl]
        if not group: continue
        lines.append(f"{'='*8} {lvl} ({len(group)}) {'='*8}")
        for i in group:
            lines.append(f"[{i.code}] {i.where}: {i.msg}")
    return "\n".join(lines)

In [3]:
tl = load_timeline("D:/ThienPV/code/demo/assets/layouts_timeline.json")
issues = validate_timeline(tl)
print(format_issues(issues))
tl
assets = collect_assets(tl)
print("Images:", assets["images"])
print("Videos:", assets["videos"])


✅ No validation issues.
Images: {'D:/ThienPV/code/demo/assets/diagram_etl.png': True, 'D:/ThienPV/code/demo/assets/chart_results.png': True}
Videos: {'D:/ThienPV/code/demo/assets/presenter.mp4': True}


# General

## utils

In [4]:

import os
from typing import Union, List, Dict, Tuple, Optional
import numpy as np
from moviepy import VideoFileClip, ImageClip, concatenate_videoclips, vfx
from PIL import Image

def get_video_resolution(path: str) -> tuple[int, int]:
    """Trả về (width, height) của video/ảnh."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    clip = VideoFileClip(path)
    w, h = clip.size
    clip.close()
    return int(w), int(h)


def top_colors_first_frame(
    video: Union[str, "VideoFileClip"],
    top_k: int = 10,
    quantize: int = 0,
    resize_to: Optional[int] = None,
    return_hex: bool = True,
) -> List[Dict]:
    """
    Lấy frame đầu (t=0) và trả về top_k màu xuất hiện nhiều nhất.

    Params
    ------
    video       : đường dẫn hoặc VideoFileClip (MoviePy v2).
    top_k       : số màu cần lấy (mặc định 10).
    quantize    : bước lượng tử hoá kênh màu (0 = đếm màu chính xác).
                  Ví dụ 16 -> gom màu theo bậc 16 (giảm nhiễu).
    resize_to   : nếu set (vd 720), sẽ downscale cạnh dài của frame
                  về <= giá trị này để tăng tốc đếm (không bắt buộc).
    return_hex  : có trả kèm mã HEX hay không.

    Returns
    -------
    List[Dict] với mỗi phần tử:
      {
        "rgb": (r, g, b),
        "hex": "#RRGGBB",   # nếu return_hex=True
        "count": <số pixel>,
        "ratio": <tỷ lệ pixel trên toàn frame>
      }
    """
    opened_here = False
    clip = video
    if isinstance(video, str):
        clip = VideoFileClip(video)
        opened_here = True
    try:
        frame = clip.get_frame(0.0)  # (H,W,3|4)
    finally:
        if opened_here:
            clip.close()

    # Chuẩn hoá về RGB uint8
    if frame.ndim != 3 or frame.shape[2] < 3:
        raise ValueError("Frame không hợp lệ (cần dạng HxWx3 hoặc HxWx4).")
    frame = frame[:, :, :3]
    if frame.dtype != np.uint8:
        # MoviePy đôi khi trả float [0..1]; chuẩn hoá về uint8
        if frame.max() <= 1.0:
            frame = np.clip(np.rint(frame * 255.0), 0, 255).astype(np.uint8)
        else:
            frame = np.clip(np.rint(frame), 0, 255).astype(np.uint8)

    # Tùy chọn downscale để tăng tốc
    if resize_to:
        h, w = frame.shape[:2]
        scale = resize_to / max(h, w) if max(h, w) > resize_to else 1.0
        if scale < 1.0:
            new_size = (int(w * scale), int(h * scale))
            frame = np.array(Image.fromarray(frame).resize(new_size, Image.BILINEAR), dtype=np.uint8)

    # Tùy chọn lượng tử hoá màu (gom cụm màu gần nhau)
    if quantize and quantize > 1:
        q = int(quantize)
        frame = (frame.astype(np.int16) // q) * q + q // 2
        frame = np.clip(frame, 0, 255).astype(np.uint8)

    # Đếm nhanh bằng numpy.unique trên dtype cấu trúc
    pixels = frame.reshape(-1, 3)
    dtype = np.dtype([("r", np.uint8), ("g", np.uint8), ("b", np.uint8)])
    structured = pixels.view(dtype)
    uniques, counts = np.unique(structured, return_counts=True)

    if counts.size == 0:
        return []

    # Lấy top_k bằng argpartition (O(n))
    k = min(top_k, counts.size)
    idx = np.argpartition(-counts, kth=k - 1)[:k]
    idx = idx[np.argsort(-counts[idx])]  # sắp xếp lại theo count giảm dần

    total = pixels.shape[0]
    results: List[Dict] = []
    for i in idx:
        r, g, b = int(uniques[i]["r"]), int(uniques[i]["g"]), int(uniques[i]["b"])
        cnt = int(counts[i])
        item: Dict[str, object] = {"rgb": (r, g, b), "count": cnt, "ratio": cnt / total}
        if return_hex:
            item["hex"] = f"#{r:02X}{g:02X}{b:02X}"
        results.append(item)

    return results


def get_input(input):
    if isinstance(input, str):
        if input.lower().endswith((".jpg", ".png", ".jpeg", ".webp")):
            return ImageClip(input)
        return VideoFileClip(input)
    return input


## Core

In [5]:
import logging
import os
from moviepy import VideoFileClip, ImageClip, concatenate_videoclips, vfx
import numpy as np
from moviepy.video.fx.MaskColor import MaskColor
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(logging.Formatter(
        '[%(levelname)s] %(asctime)s - %(name)s: %(message)s',
        datefmt='%H:%M:%S'
    ))
    logger.addHandler(console_handler)


def get_logger(name: str = __name__):
    """Trả về logger cho module khác."""
    return logging.getLogger(name)


def trim_video(input_path: str, start: float, end: float):
    """
    Cắt video từ giây `start` đến `end`.
    """
    if end <= start:
        raise ValueError("'end' phải lớn hơn 'start'.")

    logger.info(f"Trimming {input_path} from {start}s to {end}s")
    clip = VideoFileClip(input_path).subclipped(start, end)
    return clip


def concat_videos(videos: list[str|VideoFileClip]):
    """Ghép nhiều video thành một."""
    if len(videos) < 2:
        raise ValueError("Cần ít nhất 2 video để ghép.")
    clips= []
    for v in videos:
        if isinstance(v, str):
            clips.append(VideoFileClip(v))
        elif isinstance(v, VideoFileClip):
            clips.append(v) 
        else:
            raise TypeError("Only support VideoFileCLip or str")

    logger.info(f"Concatenating {len(videos)} videos")
    final_clip = concatenate_videoclips(clips, method="chain")
    return final_clip


def overlay(fg, bg,
            position: tuple[int, int] = (0, 0), relative:bool= False, scale: float = 1.0,
            bg_loop: bool = True, duration: float | None = None):
    """
    Overlay video foreground (đã xóa nền) lên background.
    """
    fg = get_input(fg)
    bg = get_input(bg)
    if duration:
        fg = fg.with_duration(duration)
        bg = bg.with_duration(duration)
    else:
        bg = bg.with_duration(fg.duration)
    if scale != 1.0:
        fg = fg.resized(scale)

    # Overlay
    final = CompositeVideoClip([bg,fg.with_position(position, relative = relative)])
    return final

# Geometry & Layout

In [22]:
# geometry.py

In [6]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Tuple, Literal
import logging

Rect = Tuple[int, int, int, int]  # (x, y, w, h)

@dataclass(frozen=True)
class Meta:
    """Thông tin khung canvas & safe margins (tỷ lệ)."""
    width: int
    height: int
    title_area: float = 0.05   # 5% mỗi cạnh
    action_area: float = 0.025 # 2.5% mỗi cạnh


In [7]:
# ---------- utils ----------
def _clamp(v: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, v))

def _round_i(v: float) -> int:
    return int(round(v))

In [8]:
# ---------- 1) snap_to_safe ----------
def snap_to_safe(rect: Rect, meta: Meta) -> Rect:
    """
    Kẹp rect vào *title-safe box* theo meta.title_area.
    Lưu ý: chỉ tịnh tiến (không co giãn). Nếu rect lớn hơn vùng title-safe,
    nó sẽ bám mép trái/trên của title-safe và có thể tràn ra ngoài title-safe,
    nhưng vẫn luôn nằm *trong canvas*.

    Mẹo: nếu muốn kẹp theo action-safe, có thể truyền Meta với title_area = action_area
    (vd: Meta(w,h,title_area=meta.action_area, action_area=meta.action_area)).

    Args:
        rect: (x, y, w, h) theo px.
        meta: thông số canvas/safe.

    Returns:
        Rect đã kẹp.
    """
    x, y, w, h = rect
    cw, ch = meta.width, meta.height

    # Tính title-safe box (theo tỷ lệ biên mỗi cạnh)
    m = meta.title_area
    safe_left   = _round_i(cw * m)
    safe_right  = cw - safe_left
    safe_top    = _round_i(ch * m)
    safe_bottom = ch - safe_top

    # Nếu rect lớn hơn vùng title-safe, ta vẫn giữ w/h
    # và chỉ đảm bảo không vượt canvas
    safe_w = max(0, safe_right - safe_left)
    safe_h = max(0, safe_bottom - safe_top)

    if w > safe_w:
        # Không thể hoàn toàn nằm trong title-safe; bám mép trái title-safe
        x_new = safe_left
        # Nhưng vẫn phải đảm bảo không vượt canvas
        x_new = _round_i(_clamp(x_new, 0, cw - w))
    else:
        x_new = _round_i(_clamp(x, safe_left, safe_right - w))

    if h > safe_h:
        y_new = safe_top
        y_new = _round_i(_clamp(y_new, 0, ch - h))
    else:
        y_new = _round_i(_clamp(y, safe_top, safe_bottom - h))

    return (x_new, y_new, _round_i(w), _round_i(h))

In [9]:
# ---------- 2) fit_into_rect ----------
def fit_into_rect(src_w: int, src_h: int, dst_rect: Rect, mode: str = "fit") -> Rect:
    """
    Tính kích thước & vị trí *giữa* dst_rect để đặt nội dung theo tỉ lệ,
    không méo hình.

    - mode="fit": toàn bộ nội dung nhìn thấy, có thể có viền trống (contain).
    - mode="cover": lấp đầy dst_rect, có thể crop phần thừa.

    Trả về rect mới (x, y, w, h) đã căn giữa trong dst_rect.

    Lưu ý: Hàm này *không* cắt clip; việc mask/crop nên do renderer xử lý
    dựa trên dst_rect nếu mode="cover".
    """
    dx, dy, dw, dh = dst_rect
    if src_w <= 0 or src_h <= 0 or dw <= 0 or dh <= 0:
        return (dx, dy, 0, 0)

    ar_src = src_w / float(src_h)
    ar_dst = dw / float(dh)

    if mode not in ("fit", "cover"):
        mode = "fit"

    if mode == "fit":
        scale = min(dw / src_w, dh / src_h)
    else:  # cover
        scale = max(dw / src_w, dh / src_h)

    w = src_w * scale
    h = src_h * scale
    # căn giữa trong dst_rect
    x = dx + (dw - w) / 2.0
    y = dy + (dh - h) / 2.0

    # làm tròn & đảm bảo không vượt hoàn toàn khỏi canvas con
    w_i = max(1, _round_i(w))
    h_i = max(1, _round_i(h))
    x_i = _round_i(x)
    y_i = _round_i(y)

    return (x_i, y_i, w_i, h_i)

In [10]:
# ---------- 3) place_in_rect ----------
def place_in_rect(dst_rect: Rect, align: Literal["left", "center", "right"]) -> Tuple[int, int]:
    """
    Trả về *điểm neo* (anchor_x, center_y) bên trong dst_rect theo căn lề ngang:
      - 'left'   -> anchor_x = left
      - 'center' -> anchor_x = center_x
      - 'right'  -> anchor_x = right

    Y luôn là tâm dọc của dst_rect (center_y).
    Khi đặt một phần tử có bề rộng w', bạn có thể tính x-left như sau:
      - align=='left'   : x_left = anchor_x
      - align=='center' : x_left = anchor_x - w'/2
      - align=='right'  : x_left = anchor_x - w'
    """
    x, y, w, h = dst_rect
    cy = _round_i(y + h / 2.0)
    if align == "center":
        ax = _round_i(x + w / 2.0)
    elif align == "right":
        ax = _round_i(x + w)
    else:
        ax = _round_i(x)
    return ax, cy


In [11]:
# ---------- 4) warn_if_upscale ----------
def warn_if_upscale(src_wh: Tuple[int, int], dst_rect: Rect, limit: float = 1.5) -> None:
    """
    Cảnh báo (logging.warning) nếu tỉ lệ phóng đại vượt ngưỡng.
    Dùng quy ước scale ~ max(dst_w/src_w, dst_h/src_h) (an toàn cho cả fit/cover).

    Args:
        src_wh: (src_w, src_h) kích thước nội dung gốc (px).
        dst_rect: (x, y, w, h) kích thước mục tiêu (px).
        limit: ngưỡng cảnh báo (mặc định 1.5 = 150%).
    """
    sw, sh = src_wh
    _, _, dw, dh = dst_rect
    if sw <= 0 or sh <= 0 or dw <= 0 or dh <= 0:
        return
    scale = max(dw / float(sw), dh / float(sh))
    if scale > limit:
        logging.warning(
            "Upscale factor %.2fx > %.2fx (src=%sx%s → dst=%sx%s). Nguy cơ mờ.",
            scale, limit, sw, sh, dw, dh
        )

In [12]:
meta = Meta(width=1920, height=1080)  # title-safe 5%

# 1) Kẹp vào title-safe
rect = (20, 20, 800, 200)
rect_safe = snap_to_safe(rect, meta)

# 2) Tính fit/cover
src_w, src_h = 1280, 720
dst = (1184, 216, 544, 408)
fit_rect   = fit_into_rect(src_w, src_h, dst, mode="fit")
cover_rect = fit_into_rect(src_w, src_h, dst, mode="cover")

# 3) Điểm neo theo align
anchor_x, center_y = place_in_rect(dst, align="right")

# 4) Cảnh báo upscale
warn_if_upscale((320, 240), dst, limit=1.5)


# Text Adapter

In [30]:
# render_text_slot(slot: Slot, meta: Meta, fonts: Fonts) -> VideoClip
# Gọi wrapped_text_clip/_mk_text_clip/make_text tuỳ theo slot_id, rect, size.common.

# render_captions(slot: Slot, meta: Meta) -> VideoClip
# Render phụ đề theo rect, viền/outlines tối thiểu, 2 dòng.

# apply_text_motion(clip: VideoClip, motion: MotionSpec | None) -> VideoClip
# Thêm enter/exit cơ bản (fade/slide).

# Presenter Handling

## Xóa nền xanh

In [17]:
from moviepy import VideoClip

def remove_green_background(src_or_clip,
                               chroma_color=(0,255,0),
                               thr: int = 40,       # 0..255
                               stiffness: int = 3   # “gắt”/độ mềm mép
                               ) -> VideoClip:
    """
    Xóa nền xanh bằng Effect API v2.
    - src_or_clip: đường dẫn video/ảnh hoặc clip.
    - chroma_color: màu xanh cần key (R,G,B).
    """
    if hasattr(src_or_clip, "get_frame"):
        clip = src_or_clip
    else:
        ext = os.path.splitext(str(src_or_clip).lower())[1]
        if ext in (".mp4",".mov",".mkv",".webm",".avi"):
            clip = VideoFileClip(src_or_clip)
        else:
            im = Image.open(src_or_clip).convert("RGB")
            clip = ImageClip(np.array(im)).with_duration(1)

    eff = MaskColor(color=chroma_color, threshold=thr, stiffness=int(stiffness)).copy()
    keyed = clip.with_effects([eff])   # keyed có .mask; áp mask tự động
    return keyed

In [14]:
presenter = VideoFileClip(filename="D:/ThienPV/code/demo/assets/presenterv1.mp4")
presenter_1 = trim_video(input_path="D:/ThienPV/code/demo/assets/presenterv1.mp4", start=0, end=04.040)
presenter_2 = trim_video(input_path="D:/ThienPV/code/demo/assets/presenterv1.mp4", start=06.483,end=08.5830)

[INFO] 09:57:00 - __main__: Trimming D:/ThienPV/code/demo/assets/presenterv1.mp4 from 0s to 4.04s
INFO:__main__:Trimming D:/ThienPV/code/demo/assets/presenterv1.mp4 from 0s to 4.04s
[INFO] 09:57:00 - __main__: Trimming D:/ThienPV/code/demo/assets/presenterv1.mp4 from 6.483s to 8.583s
INFO:__main__:Trimming D:/ThienPV/code/demo/assets/presenterv1.mp4 from 6.483s to 8.583s


In [15]:
color = top_colors_first_frame(video=presenter_1)
color[0]

{'rgb': (37, 211, 141),
 'count': 486877,
 'ratio': 0.41741855281207135,
 'hex': '#25D38D'}

In [18]:
keyed = remove_green_background(presenter_1, chroma_color=top_colors_first_frame(presenter_1)[0]["rgb"])

## Vẽ khung tròn

In [19]:
# circle_avatar_fix.py
from typing import Tuple
import numpy as np
from PIL import Image, ImageDraw
from moviepy import vfx, CompositeVideoClip, ImageClip, ColorClip
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.VideoClip import VideoClip

Rect = Tuple[int,int,int,int]

def _square_center_crop(clip: VideoClip, face_bias: float = 0.45) -> VideoClip:
    W, H = clip.w, clip.h
    if W == H: return clip
    side = min(W, H)
    x1 = (W - side) / 2
    y_center = H * face_bias
    y1 = max(0, min(H - side, y_center - side/2))
    return clip.fx(vfx.crop, x1=x1, y1=y1, width=side, height=side)

def _circle_mask(d: int):
    """Mask tròn 0..1 (ismask=True)."""
    im = Image.new("L", (d, d), 0)
    ImageDraw.Draw(im).ellipse([0,0,d,d], fill=255)
    arr = (np.array(im).astype("float32") / 255.0)
    return ImageClip(arr, is_mask=True)

def make_circle_avatar(src: str, rect: Rect, canvas_size=(1920,1080),
                       with_bg: bool = True, face_bias: float = 0.45,
                       bg_opacity: float = 0.5) -> VideoClip:
    """
    Video -> avatar tròn đặt vào rect trên canvas_size.
    - rect = (x,y,w,h), thực tế D = min(w,h).
    - with_bg: thêm vòng tròn đen mờ phía sau (mask tròn + opacity).
    """
    x, y, w, h = rect
    D = int(min(w, h))

    base = VideoFileClip(src)
    dur  = base.duration

    # 1) crop vuông + resize về D×D
    sq   = _square_center_crop(base, face_bias=face_bias).resized((D, D))

    # 2) mask tròn cho avatar
    mask = _circle_mask(D).with_duration(dur)
    avatar = sq.with_mask(mask).with_position((x, y))

    layers = []

    # 3) nền tròn mờ (ColorClip đen + mask tròn + opacity)
    if with_bg:
        black = ColorClip((D, D), color=(0,0,0)).with_duration(dur).with_opacity(bg_opacity)
        black = black.with_mask(mask)                # giới hạn trong hình tròn
        black = black.with_position((x, y))
        layers.append(black)

    layers.append(avatar)

    # 4) trả về composite kích thước canvas (quan trọng để preview không “đen”)
    return CompositeVideoClip(layers, size=canvas_size)


In [20]:
from moviepy import CompositeVideoClip, ColorClip

SRC = r"D:/ThienPV/code/demo/assets/presenterv1.mp4"

bg = ColorClip((1920,1080), color=(24,28,36)).with_duration(5)
avatar = make_circle_avatar(SRC, rect=(1440,720,360,360),
                            canvas_size=(1920,1080),
                            with_bg=True, bg_opacity=0.5)

final = CompositeVideoClip([bg, avatar], size=(1920,1080)).with_duration(5)
# final.write_videofile("avatar_circle_demo.mp4", fps=30, codec="libx264", audio=True)


# Graphics

In [21]:
# renderers.py
from __future__ import annotations
import os, io, logging, math
from dataclasses import dataclass
from typing import Optional, Tuple, Literal

import numpy as np
from PIL import Image

from moviepy import (
    ImageClip, VideoFileClip, CompositeVideoClip, ColorClip, vfx, VideoClip
)

# ====== Dataclasses khớp spec rút gọn ======
@dataclass
class GraphicSpec:
    src: str
    rect: Rect                       # (x, y, w, h) đích
    mode: Literal["fit","cover"] = "fit"
    caption: Optional[str] = None    # nếu có: thêm caption panel phía dưới
    layer: int = 5

@dataclass
class PresenterSpec:
    src: str                         # video/ảnh presenter
    rect: Rect
    shape: Literal["circle","rect"] = "circle"
    layer: int = 20
    with_bg: bool = True             # circle: thêm nền mờ phía sau


In [33]:
def _rasterize_svg_to_pil(svg_path: str, dpi: int = 180) -> Image.Image:
    """
    Chuyển SVG -> PIL.Image (RGBA). Cần cairosvg. Nếu không có, raise rõ ràng.
    """
    try:
        import cairosvg
    except Exception as e:
        raise RuntimeError("Cần 'cairosvg' để rasterize SVG: pip install cairosvg") from e

    with open(svg_path, "rb") as f:
        svg_bytes = f.read()
    png_bytes = cairosvg.svg2png(bytestring=svg_bytes, dpi=dpi)
    return Image.open(io.BytesIO(png_bytes)).convert("RGBA")

def _load_image_to_pil(path: str) -> Image.Image:
    ext = os.path.splitext(path)[1].lower()
    if ext == ".svg":
        im = _rasterize_svg_to_pil(path)
    else:
        im = Image.open(path).convert("RGBA")
    return im

def _pil_to_imageclip(im: Image.Image) -> ImageClip:
    arr = np.array(im)  # HxWx(3|4)
    clip = ImageClip(arr, transparent=True)
    return clip


In [22]:
# ====== 1) render_graphic ======
def render_graphic(gfx: GraphicSpec, meta: Meta) -> VideoClip:
    """
    Đọc ảnh (PNG/JPG/SVG), scale theo tỉ lệ và đặt vào gfx.rect.
    - mode='fit': contain không crop. 'cover': lấp đầy, có thể crop (renderer cắt theo dst_rect).
    - Nếu có caption: thêm panel mờ + TextClip phía dưới.
    Ghi chú: clip trả về có duration mặc định 1s -> caller nên .with_duration(scene.duration).
    """
    if not os.path.exists(gfx.src) and not gfx.src.startswith("data:"):
        logging.warning("Graphic src không tồn tại: %s", gfx.src)

    # 1) Tải ảnh & kích thước nguồn
    if gfx.src.startswith("data:"):
        # Data URI -> đọc bằng PIL
        import base64, re
        m = re.match(r"data:image/(png|jpeg);base64,(.*)", gfx.src, re.IGNORECASE)
        if not m:
            raise ValueError("Data URI không hỗ trợ (chỉ png/jpeg base64).")
        ext, b64 = m.groups()
        data = base64.b64decode(b64)
        im = Image.open(io.BytesIO(data)).convert("RGBA")
    else:
        im = _load_image_to_pil(gfx.src)
    sw, sh = im.size

    # 2) Tính rect "fit" hoặc "cover" bên trong vùng đích
    dst_rect = snap_to_safe(gfx.rect, meta)
    fitted = fit_into_rect(sw, sh, dst_rect, mode=gfx.mode)

    # 3) Cảnh báo upscale (nếu quá 150%)
    warn_if_upscale((sw, sh), fitted, limit=1.5)

    # 4) Tạo ImageClip & resize về fitted, đặt position
    img_clip = _pil_to_imageclip(im).resized((fitted[2], fitted[3])).with_position((fitted[0], fitted[1]))

    # 5) Caption (nếu có)
    if gfx.caption:
        # panel mờ + caption text
        cap_x, cap_y = dst_rect[0], dst_rect[1] + dst_rect[3] + 24
        cap_w, cap_h = dst_rect[2], 56
        panel = ColorClip(size=(cap_w, cap_h), color=(0,0,0)).with_opacity(0.55).with_position((cap_x, cap_y))
        try:
            # TextClip thường cần ImageMagick; nếu môi trường không có, hãy thay bằng text tools riêng của bạn
            from moviepy import TextClip
            txt = TextClip(gfx.caption, fontsize=28, color="white", font="Inter", method="caption", size=(cap_w-24, cap_h-16), align="center")
            txt = txt.with_position((cap_x+12, cap_y+8))
            cap = CompositeVideoClip([panel, txt], size=(meta.width, meta.height))
        except Exception:
            logging.warning("Không thể tạo TextClip cho caption (thiếu ImageMagick?). Chỉ hiển thị panel.")
            cap = CompositeVideoClip([panel], size=(meta.width, meta.height))
        out = CompositeVideoClip([img_clip, cap], size=(meta.width, meta.height))
    else:
        out = CompositeVideoClip([img_clip], size=(meta.width, meta.height))

    return out.with_duration(1)  # caller override bằng scene.duration


In [23]:
from __future__ import annotations
import os, io, logging
from dataclasses import dataclass
from typing import Optional, Tuple, Literal
import numpy as np
from PIL import Image

from moviepy import (
    ImageClip, VideoFileClip, CompositeVideoClip, ColorClip, VideoClip, vfx
)


In [24]:

def _make_circle_rgba(w: int, h: int, color=(0, 0, 0, 115)) -> Image.Image:
    from PIL import ImageDraw, ImageFilter
    im = Image.new("RGBA", (w, h), (0,0,0,0))
    draw = ImageDraw.Draw(im, "RGBA")
    r = min(w, h); x0=(w-r)//2; y0=(h-r)//2; x1=x0+r; y1=y0+r
    draw.ellipse([x0,y0,x1,y1], fill=color)
    return im.filter(ImageFilter.GaussianBlur(radius=max(1, r//80)))

def _make_circle_mask_clip(w: int, h: int) -> ImageClip:
    from PIL import ImageDraw
    im = Image.new("L", (w, h), 0)
    draw = ImageDraw.Draw(im)
    r = min(w, h); x0=(w-r)//2; y0=(h-r)//2; x1=x0+r; y1=y0+r
    draw.ellipse([x0,y0,x1,y1], fill=255)
    return ImageClip(np.array(im), is_mask=True)

In [25]:
def _resize_clip(clip: VideoClip, size: Tuple[int,int]) -> VideoClip:
    # v2: .resized ; v1: .fx(vfx.resize, ...)
    if hasattr(clip, "resized"):
        return clip.resized(size)
    return clip.fx(vfx.resize, size)

def _set_position(clip: VideoClip, pos) -> VideoClip:
    if hasattr(clip, "with_position"):  # v2
        return clip.with_position(pos)
    if hasattr(clip, "set_position"):   # v1
        return clip.set_position(pos)
    return clip.set_pos(pos)            # legacy alias

def _set_duration(clip: VideoClip, dur: float) -> VideoClip:
    if hasattr(clip, "with_duration"):
        return clip.with_duration(dur)
    return clip.set_duration(dur)

def _set_mask(clip: VideoClip, mask: VideoClip) -> VideoClip:
    if hasattr(clip, "with_mask"):
        return clip.with_mask(mask)
    return clip.set_mask(mask)

In [26]:
def _square_center_crop(clip: VideoClip, face_bias: float = 0.45) -> VideoClip:
    """
    Crop clip về hình vuông 1:1. face_bias ~ 0..1: 0.5 = tâm, 0.45 = hơi dồn lên (giữ nhiều phần đầu/ngực).
    """
    W, H = clip.w, clip.h
    if W == H:
        return clip
    side = min(W, H)
    x_center = W / 2.0
    y_center = H * face_bias
    return clip.fx(vfx.crop, x_center=x_center, y_center=y_center, width=side, height=side)

In [27]:
def _apply_chroma_key_green(clip: VideoClip,
                            color=(0, 255, 0),
                            thr=40, s=3) -> VideoClip:
    """
    Xóa nền xanh. Tham số thr, s phụ thuộc phiên bản MoviePy:
    - Nhiều bản chấp nhận thr (0..255) & softness s (int).
    - Nếu môi trường yêu cầu (0..1), bạn có thể thử thr/255.0.
    """
    try:
        keyed = clip.fx(vfx.mask_color, color=color, thr=thr, s=s)
        return _set_mask(clip, keyed.mask)  # áp mask vừa tạo
    except Exception as e:
        logging.warning("mask_color thất bại (%s). Dùng fallback đơn giản.", e)
        # Fallback rất đơn giản (ít chính xác): hard threshold green
        def make_mask(get_frame, t):
            f = get_frame(t).astype(np.int16)
            r, g, b = f[...,0], f[...,1], f[...,2]
            # xanh mạnh khi g lớn hơn r/b rõ rệt
            m = (g - np.maximum(r, b)) > 50
            return (m * 255).astype("uint8")
        maskclip = clip.image_transform(lambda fr: make_mask(lambda _: fr, 0), apply_to=[])
        maskclip.ismask = True
        return _set_mask(clip, maskclip)

In [28]:
# ====== Dataclass khớp spec ======
@dataclass
class PresenterSpec:
    src: str
    rect: Tuple[int,int,int,int]
    shape: Literal["circle","rect"] = "circle"   # circle: mask tròn; rect: giữ vuông, xóa nền xanh
    layer: int = 20
    with_bg: bool = True                         # circle: thêm nền mờ dưới


In [35]:


# ====== render_presenter (đã nâng cấp) ======
def render_presenter(pip: PresenterSpec) -> VideoClip:
    """
    Avatar 1:1 từ ngực trở lên, 2 dạng:
      - shape='circle': input có nền → crop 1:1, thêm nền tròn mờ, mask tròn.
      - shape='rect'  : input nền xanh → chroma-key xóa nền, giữ hình vuông.
    """
    x, y, w, h = pip.rect
    ext = os.path.splitext(pip.src)[1].lower()
    is_video = ext in (".mp4", ".mov", ".mkv", ".webm", ".avi")

    # 1) Tải source
    if is_video:
        base = VideoFileClip(pip.src)
        src_dur = base.duration
    else:
        im = Image.open(pip.src).convert("RGBA")
        base = _pil_to_imageclip(im)
        src_dur = 1

    # 2) Đưa về 1:1 (center crop hơi dồn lên đầu 45%)
    # base_sq = _square_center_crop(base, face_bias=0.45)
    base_sq = base
    layers = []

    if pip.shape == "circle":
        # 3a) Thêm nền tròn mờ (nếu cần) trước avatar
        if pip.with_bg:
            bg_rgba = _make_circle_rgba(w, h, color=(0,0,0,120))
            bg_clip = _pil_to_imageclip(bg_rgba)
            bg_clip = _set_position(bg_clip, (x, y))
            bg_clip = _set_duration(bg_clip, src_dur)
            layers.append(bg_clip)

        # 3b) Resize + mask tròn
        avatar = _resize_clip(base_sq, (w, h))
        mask   = _make_circle_mask_clip(w, h)
        avatar = _set_mask(avatar, mask)
        avatar = _set_position(avatar, (x, y))
        layers.append(avatar)

    else:  # shape == 'rect' (green background → remove)
        # 3c) Chroma-key nền xanh
        keyed = _apply_chroma_key_green(base_sq, color=(0,255,0), thr=40, s=3)
        # 3d) Resize + đặt vị trí (giữ góc vuông, alpha từ key)
        avatar = _resize_clip(keyed, (w, h))
        avatar = _set_position(avatar, (x, y))
        layers.append(avatar)

    comp = CompositeVideoClip(layers)

    # 4) Duration:
    if is_video:
        return comp  # giữ duration gốc video
    else:
        # ảnh tĩnh: để 1s, caller sẽ .with_duration(...) khi ráp scene
        return _set_duration(comp, 1)

In [36]:
# mock_presenter_demo.py
from moviepy import CompositeVideoClip, ColorClip
from moviepy.video.io.VideoFileClip import VideoFileClip
import os, logging

logging.basicConfig(level=logging.INFO)

SRC = r"D:/ThienPV/code/demo/assets/presenterv1.mp4"
OUT = "demo_presenter_mock.mp4"
DUR = 5

# Khung nền để nhìn rõ avatar
bg = ColorClip(size=(1920, 1080), color=(18, 22, 30)).with_duration(DUR)

def safe_presenter(pip: PresenterSpec):
    """Gọi render_presenter, nếu lỗi (file không tồn tại/codec) thì trả về ô màu mock."""
    try:
        clip = render_presenter(pip)
        # ép về 5s để không kéo dài timeline
        if hasattr(clip, "with_duration"):
            return clip.with_duration(DUR)
        return clip.set_duration(DUR)
    except Exception as e:
        logging.warning("Không render được presenter (%s). Dùng mock clip thay thế.", e)
        x, y, w, h = pip.rect
        mock = ColorClip(size=(w, h), color=(60, 90, 140))
        if hasattr(mock, "with_duration"):
            mock = mock.with_duration(DUR)
        else:
            mock = mock.set_duration(DUR)
        if hasattr(mock, "with_position"):
            mock = mock.with_position((x, y))
        else:
            mock = mock.set_position((x, y))
        return mock

# 1) Avatar tròn (mask tròn + nền mờ)
pip_circle = PresenterSpec(
    src=SRC,
    rect=(1440, 640, 360, 360),  # góc phải dưới
    shape="circle",
    with_bg=True
)
clip_circle = safe_presenter(pip_circle)

# 2) Avatar vuông (xóa nền xanh)
pip_rect = PresenterSpec(
    src=SRC,
    rect=(1040, 740, 420, 236),  # khung chữ nhật (ví dụ demo)
    shape="rect",                # sẽ chroma-key green
    with_bg=False
)
clip_rect = safe_presenter(pip_rect)

final = CompositeVideoClip([bg, clip_circle, clip_rect], size=(1920, 1080))

# Xuất thử (bật dòng dưới khi sẵn sàng)
# final.write_videofile(OUT, fps=30, codec="libx264", audio=True)
print("Ready! Bật dòng write_videofile để xuất:", OUT)


Ready! Bật dòng write_videofile để xuất: demo_presenter_mock.mp4


In [37]:
from moviepy import CompositeVideoClip, ColorClip
meta = Meta(1920,1080, title_area=0.05, action_area=0.025)

# Nền trơn (tùy, có thể bỏ)
bg = ColorClip(size=(1920,1080), color=(13,17,23)).with_duration(5)

# Ảnh nền: cover toàn khung, KHÔNG Ken Burns
photo = render_graphic(
    GraphicSpec(
        src=r"D:/ThienPV/code/demo/assets/img.jpg",
        rect=(0,0,1920,1080),
        mode="cover"
    ),
    meta
).with_duration(5)

# Graphic (fit + caption)
gclip = render_graphic(
    GraphicSpec(
        src=r"D:/ThienPV/code/demo/assets/diagram_etl.png",
        rect=(1184,216,544,408),
        mode="fit",
        caption="Sơ đồ ETL (fit/contain, không crop)."
    ),
    meta
).with_duration(5)

# Presenter (cắt tròn + nền mờ)
pclip = render_presenter(
    PresenterSpec(
        src=r"D:/ThienPV/code/demo/assets/presenter.mp4",
        rect=(1440,720,360,360),
        shape="circle",
        with_bg=True
    )
).with_duration(5)

final = CompositeVideoClip([bg, photo, gclip, pclip], size=(1920,1080))
# final.write_videofile("demo.mp4", fps=30, codec="libx264", audio=False)


In [38]:
final.preview()

OSError: [Errno 22] Invalid argument

MoviePy error: FFPLAY encountered the following error while previewing clip :

 None

In [ ]:
# render_graphic(gfx: GraphicSpec, meta: Meta) -> VideoClip
# Đọc ảnh/SVG/PNG, “fit” vào layout.rect, thêm caption nếu có.

# render_presenter(pip: PresenterSpec) -> VideoClip
# Cắt khung tròn (mask) hoặc giữ khung chữ nhật, đặt đúng rect.

# ken_burns(clip: VideoClip, scale_to: float = 1.06, dur: float = 4.0) -> VideoClip
# Ken‑Burns nhẹ cho ảnh nền/ảnh minh hoạ (tùy chọn).

# Scene & Timeline

In [ ]:
# render_scene(scene: Scene, meta: Meta, assets: dict) -> VideoClip
# Lớp nền (màu/ảnh/video) → graphics → text slots → presenter → UI (progress/timestamp/captions).

# add_transition(prev: VideoClip, nxt: VideoClip, kind: str = "cut", dur: float = 0.3) -> VideoClip
# Hỗ trợ cut và dissolve_short (crossfade).

# assemble_timeline(tl: Timeline, assets: dict) -> list[VideoClip]
# Render từng scene theo start/duration, trả về danh sách clip đã căn thời lượng.

# stitch_with_transitions(clips: list[VideoClip], tl: Timeline) -> VideoClip
# Nối các clip và áp transition_out của mỗi scene.render_scene(scene: Scene, meta: Meta, assets: dict) -> VideoClip
# Lớp nền (màu/ảnh/video) → graphics → text slots → presenter → UI (progress/timestamp/captions).

# add_transition(prev: VideoClip, nxt: VideoClip, kind: str = "cut", dur: float = 0.3) -> VideoClip
# Hỗ trợ cut và dissolve_short (crossfade).

# assemble_timeline(tl: Timeline, assets: dict) -> list[VideoClip]
# Render từng scene theo start/duration, trả về danh sách clip đã căn thời lượng.

# stitch_with_transitions(clips: list[VideoClip], tl: Timeline) -> VideoClip
# Nối các clip và áp transition_out của mỗi scene.

# Audio

In [ ]:
# normalize_lufs(wav_path: str, target: float = -14.0) -> str
# Chuẩn hoá loudness về −14 LUFS (có thể call ffmpeg loudnorm).

# mix_audio(voice: str | None, music: str | None, ducking_db: float = -20.0) -> AudioFileClip
# Trộn voice với nhạc nền (nhạc nhỏ hơn ~20 dB).

# attach_audio(clip: VideoClip, audio: AudioFileClip) -> VideoClip

# Export & Preview

In [ ]:
# export_video(clip: VideoClip, out_path: str, codec: str = "libx264", crf: int = 18, fps: int | None = None) -> None
# Xuất MP4 1080p H.264/AAC (mezzanine/preview tuỳ tham số).

# export_preview_gif(clip: VideoClip, out_path: str, fps: int = 12, scale: float = .5) -> None
# Xuất GIF ngắn để review layout.